# Additional Code Processing
This notebook contains additional processing of the variety trial codes to construct the parameter set for model scoring.

In [32]:
# Import Libraries
import numpy as np
import pandas as pd

In [33]:
# Read in the raw codes csv file
df_raw = pd.read_csv('codes_all.csv')
print(df_raw.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 34 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   code       84 non-null     object 
 1   trial_001  84 non-null     float64
 2   trial_006  84 non-null     float64
 3   trial_008  84 non-null     float64
 4   trial_009  84 non-null     float64
 5   trial_011  84 non-null     float64
 6   trial_013  84 non-null     float64
 7   trial_015  84 non-null     float64
 8   trial_016  84 non-null     float64
 9   trial_018  84 non-null     float64
 10  trial_023  84 non-null     float64
 11  trial_026  84 non-null     float64
 12  trial_034  84 non-null     float64
 13  trial_036  84 non-null     float64
 14  trial_040  84 non-null     float64
 15  trial_044  84 non-null     float64
 16  trial_045  84 non-null     float64
 17  trial_048  84 non-null     float64
 18  trial_052  84 non-null     float64
 19  trial_055  84 non-null     float64
 20  trial_059  8

In [34]:
# transpose the dataframe
df_raw.drop('entity', axis=1, inplace=True)
df_raw.set_index('code', inplace=True)
df_tr = df_raw.T

In [35]:
print(df_tr.info())

<class 'pandas.core.frame.DataFrame'>
Index: 32 entries, trial_001 to trial_094
Data columns (total 84 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   bed dimensions                                  32 non-null     float64
 1   rows per bed                                    32 non-null     float64
 2   cultivar growth pattern                         32 non-null     float64
 3   cultivar name                                   32 non-null     float64
 4   fertilizer application date                     32 non-null     float64
 5   fertilizer application frequency                32 non-null     float64
 6   fertilizer application method                   32 non-null     float64
 7   fertilizer application range                    32 non-null     float64
 8   fertilizer application rate                     32 non-null     float64
 9   fertilizer brand name              

## Complex Parameter Calculation
Some parameters are complex--meaning they can be calculated as a sum or product of other parameters. For example, plant density can be calculated by multiplying plant and row spacing, or by dividing plants per plot by plot dimensions. If a complex parameter is not reported, but enough information is present to calculate it, then the parameter should be counted as reported.

In [36]:
# Determine how many trials report plant density
print(df_tr['plant density'].value_counts())

# Determine how many trials report between-row spacing and within-row spacing, but not plant density
qdf = df_tr.query('`between-row spacing` == 1 and `within-row spacing` == 1 and `plant density` == 0')
print(len(qdf))
print(qdf.index)

# Determine how many trials report within-row spacing and bed dimensions, but not plant density or rows per bed
qdf = df_tr.query('`within-row spacing` == 1 and `bed dimensions` == 1 and `plant density` == 0 and `rows per bed` == 0')
print(len(qdf))
print(qdf.index)

# Determine how many trials report plants per plot and plot dimensions, but not plant density
qdf = df_tr.query('`plants per plot` == 1 and `plot dimensions` == 1 and `plant density` == 0')
print(len(qdf))
print(qdf.index)

plant density
0.0    20
1.0    12
Name: count, dtype: int64
9
Index(['trial_013', 'trial_052', 'trial_055', 'trial_059', 'trial_060',
       'trial_066', 'trial_072', 'trial_081', 'trial_083'],
      dtype='object')
6
Index(['trial_015', 'trial_044', 'trial_048', 'trial_055', 'trial_059',
       'trial_083'],
      dtype='object')
5
Index(['trial_044', 'trial_045', 'trial_048', 'trial_052', 'trial_059'], dtype='object')


In [37]:
# Add plant density if component parameters are present
qdf1 = df_tr.query('`between-row spacing` == 1 and `within-row spacing` == 1 and `plant density` == 0')
qdf2 = df_tr.query('`within-row spacing` == 1 and `bed dimensions` == 1 and `plant density` == 0 and `rows per bed` == 0')
qdf3 = df_tr.query('`plants per plot` == 1 and `plot dimensions` == 1 and `plant density` == 0')
# merge dataframes
qdf = qdf1.merge(qdf2, how='outer', left_index=True, right_index=True)
qdf = qdf.merge(qdf3, how='outer', left_index=True, right_index=True)
# add plant density in df_tr if in qdf
for index in qdf.index:
    df_tr.loc[index, 'plant density'] = 1
print(df_tr['plant density'].value_counts())

plant density
1.0    25
0.0     7
Name: count, dtype: int64


Note: bed dimensions refers to both the width of the bed and also the distance between bed centers, which is the same as between-row spacing when rows per bed is not reported. This is because rows per bed are assumed to be 1 unless otherwise reported. If there are multiple rows per bed, then between-row and bed dimensions must be listed because they are not synonymous in this case.

In [38]:
# Determine how many trials report bed dimensions but not between-row spacing or rows per bed
qdf = df_tr.query('`bed dimensions` == 1 and `between-row spacing` == 0 and `rows per bed` == 0')
print(len(qdf))
print(qdf.index)
# Add between-row spacing if row dimensions are present
for index in qdf.index:
    df_tr.loc[index, 'between-row spacing'] = 1
print(df_tr['between-row spacing'].value_counts())

6
Index(['trial_015', 'trial_044', 'trial_048', 'trial_078', 'trial_091',
       'trial_092'],
      dtype='object')
between-row spacing
1.0    24
0.0     8
Name: count, dtype: int64


In [39]:
# Determine how many trials report between-row spacing but not bed dimensions or rows per bed
qdf = df_tr.query('`between-row spacing` == 1 and `bed dimensions` == 0 and `rows per bed` == 0')
print(len(qdf))
print(qdf.index)
# Add bed dimensions if row dimensions are present
for index in qdf.index:
    df_tr.loc[index, 'bed dimensions'] = 1
print(df_tr['bed dimensions'].value_counts())

4
Index(['trial_023', 'trial_052', 'trial_066', 'trial_081'], dtype='object')
bed dimensions
1.0    27
0.0     5
Name: count, dtype: int64


Note: fertilizer brand name implies both the fertilizer type and concentration (NPK value). This is different from fertilizer application rate, which states how much of it or its component nutrients were applied per unit area.

In [40]:
# Determine how many trials report fertilizer brand name, but not type or concentration
qdf = df_tr.query('`fertilizer brand name` == 1 and `fertilizer type` == 0 or `fertilizer concentration` == 0')
print(len(qdf))
print(qdf.index)
# add fertilizer type and concentration if brand name is present
for index in qdf.index:
    df_tr.loc[index, 'fertilizer type'] = 1
    df_tr.loc[index, 'fertilizer concentration'] = 1
print(df_tr['fertilizer type'].value_counts())

6
Index(['trial_006', 'trial_008', 'trial_034', 'trial_069', 'trial_072',
       'trial_086'],
      dtype='object')
fertilizer type
1.0    31
0.0     1
Name: count, dtype: int64


Note: the use of a -cide chemical to control fungi, pests, or weeds denotes a control method. So if a -cide is reported but control method is not, the control method should be reported.

In [41]:
# Determine how many trials report fungicide brand name, but not fungi control method
qdf = df_tr.query('`fungicide brand name` == 1 and `fungi control method` == 0')
print(len(qdf))
print(qdf.index)
# add fungi control method if brand name is present
for index in qdf.index:
    df_tr.loc[index, 'fungi control method'] = 1
print(df_tr['fungi control method'].value_counts())

0
Index([], dtype='object')
fungi control method
1.0    17
0.0    15
Name: count, dtype: int64


In [42]:
# Determine how many trials report herbicide brand name, but not weed control method
qdf = df_tr.query('`herbicide brand name` == 1 and `weed control method` == 0')
print(len(qdf))
print(qdf.index)
# add weed control method if brand name is present
for index in qdf.index:
    df_tr.loc[index, 'weed control method'] = 1
print(df_tr['weed control method'].value_counts())

0
Index([], dtype='object')
weed control method
1.0    20
0.0    12
Name: count, dtype: int64


In [43]:
# Determine how many trials report pesticide brand name, but not pest control method
qdf = df_tr.query('`pesticide brand name` == 1 and `pest control method` == 0')
print(len(qdf))
print(qdf.index)
# add pest control method if brand name is present
for index in qdf.index:
    df_tr.loc[index, 'pest control method'] = 1
print(df_tr['pest control method'].value_counts())

0
Index([], dtype='object')
pest control method
1.0    25
0.0     7
Name: count, dtype: int64


## Order of Specificity
Some parameters are more specific than others with regard to temporal or spatial precision. For example, harvest date is more specific than harvest frequency, which is in turn more specific than harvest range. Harvest date provides the exact dates of harvest, while harvest frequency provides an interval within a range, and harvest range provides just the range of dates. As harvest timing is a single parameter, its units of precision should be viewed as mutually exclusive for scoring purposes. Therefore, only one level of specificity per parameter per trial is allowed.

In [44]:
# Determine how many trials report fertilizer application frequency and range
qdf = df_tr.query('`fertilizer application frequency` == 1 and `fertilizer application range` == 1')
print(len(qdf))
print(qdf.index)
# If both are present, remove range
for index in qdf.index:
    df_tr.loc[index, 'fertilizer application range'] = 0
print(df_tr['fertilizer application range'].value_counts())

1
Index(['trial_072'], dtype='object')
fertilizer application range
0.0    32
Name: count, dtype: int64


In [45]:
# Determine how many trials report fertilizer application date and frequency
qdf = df_tr.query('`fertilizer application date` == 1 and `fertilizer application frequency` == 1')
print(len(qdf))
print(qdf.index)
# If both are present, remove frequency
for index in qdf.index:
    df_tr.loc[index, 'fertilizer application frequency'] = 0
print(df_tr['fertilizer application frequency'].value_counts())

9
Index(['trial_036', 'trial_040', 'trial_052', 'trial_066', 'trial_067',
       'trial_078', 'trial_083', 'trial_091', 'trial_094'],
      dtype='object')
fertilizer application frequency
0.0    31
1.0     1
Name: count, dtype: int64


In [46]:
# Determine how many trials report fertilizer application date and range
qdf = df_tr.query('`fertilizer application date` == 1 and `fertilizer application range` == 1')
print(len(qdf))
print(qdf.index)
# If both are present, remove range
for index in qdf.index:
    df_tr.loc[index, 'fertilizer application range'] = 0
print(df_tr['fertilizer application range'].value_counts())

0
Index([], dtype='object')
fertilizer application range
0.0    32
Name: count, dtype: int64


In [47]:
# Determine how many trials report harvest frequency and range
qdf = df_tr.query('`harvest frequency` == 1 and `harvest range` == 1')
print(len(qdf))
print(qdf.index)
# If both are present, remove range
for index in qdf.index:
    df_tr.loc[index, 'harvest range'] = 0
print(df_tr['harvest range'].value_counts())

3
Index(['trial_072', 'trial_083', 'trial_094'], dtype='object')
harvest range
0.0    29
1.0     3
Name: count, dtype: int64


In [48]:
# Determine how many trials report harvest date and frequency
qdf = df_tr.query('`harvest date` == 1 and `harvest frequency` == 1')
print(len(qdf))
print(qdf.index)
# If both are present, remove frequency
for index in qdf.index:
    df_tr.loc[index, 'harvest frequency'] = 0
print(df_tr['harvest frequency'].value_counts())

9
Index(['trial_008', 'trial_023', 'trial_026', 'trial_044', 'trial_052',
       'trial_059', 'trial_078', 'trial_083', 'trial_094'],
      dtype='object')
harvest frequency
0.0    30
1.0     2
Name: count, dtype: int64


In [49]:
# Determine how many trials report harvest date and range
qdf = df_tr.query('`harvest date` == 1 and `harvest range` == 1')
print(len(qdf))
print(qdf.index)
# If both are present, remove range
for index in qdf.index:
    df_tr.loc[index, 'harvest range'] = 0
print(df_tr['harvest range'].value_counts())

1
Index(['trial_092'], dtype='object')
harvest range
0.0    30
1.0     2
Name: count, dtype: int64


In [50]:
# Determine how many trials report fungicide application date and frequency
qdf = df_tr.query('`fungicide application date` == 1 and `fungicide application frequency` == 1')
print(len(qdf))
print(qdf.index)
# If both are present, remove frequency
for index in qdf.index:
    df_tr.loc[index, 'fungicide application frequency'] = 0
print(df_tr['fungicide application frequency'].value_counts())

0
Index([], dtype='object')
fungicide application frequency
0.0    30
1.0     2
Name: count, dtype: int64


In [51]:
# Determine how many trials report total plant biomass on sampling date and end of season
qdf = df_tr.query('`total plant biomass on sampling date` == 1 and `total plant biomass end of season` == 1')
print(len(qdf))
print(qdf.index)
# If both are present, remove end of season
for index in qdf.index:
    df_tr.loc[index, 'total plant biomass end of season'] = 0
print(df_tr['total plant biomass end of season'].value_counts())

1
Index(['trial_001'], dtype='object')
total plant biomass end of season
0.0    32
Name: count, dtype: int64


In [52]:
# Determine how many trials report harvested fruit number and cumulative harvested fruit number
qdf = df_tr.query('`harvested fruit number` == 1 and `cumulative harvested fruit number` == 1')
print(len(qdf))
print(qdf.index)
# If both are present, remove cumulative
for index in qdf.index:
    df_tr.loc[index, 'cumulative harvested fruit number'] = 0
print(df_tr['cumulative harvested fruit number'].value_counts())

2
Index(['trial_055', 'trial_059'], dtype='object')
cumulative harvested fruit number
1.0    17
0.0    15
Name: count, dtype: int64


In [53]:
# Determine how many trials report harvested  yield and cumulative harvested yield
qdf = df_tr.query('`harvested yield` == 1 and `cumulative harvested yield` == 1')
print(len(qdf))
print(qdf.index)
# If both are present, remove cumulative
for index in qdf.index:
    df_tr.loc[index, 'cumulative harvested yield'] = 0
print(df_tr['cumulative harvested yield'].value_counts())

3
Index(['trial_055', 'trial_059', 'trial_083'], dtype='object')
cumulative harvested yield
1.0    26
0.0     6
Name: count, dtype: int64


In [54]:
# transpose the dataframe
df_pr = df_tr.T
df_pr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 84 entries, bed dimensions to harvested yield
Data columns (total 32 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   trial_001  84 non-null     float64
 1   trial_006  84 non-null     float64
 2   trial_008  84 non-null     float64
 3   trial_009  84 non-null     float64
 4   trial_011  84 non-null     float64
 5   trial_013  84 non-null     float64
 6   trial_015  84 non-null     float64
 7   trial_016  84 non-null     float64
 8   trial_018  84 non-null     float64
 9   trial_023  84 non-null     float64
 10  trial_026  84 non-null     float64
 11  trial_034  84 non-null     float64
 12  trial_036  84 non-null     float64
 13  trial_040  84 non-null     float64
 14  trial_044  84 non-null     float64
 15  trial_045  84 non-null     float64
 16  trial_048  84 non-null     float64
 17  trial_052  84 non-null     float64
 18  trial_055  84 non-null     float64
 19  trial_059  84 non-null     floa

In [55]:
# print the number of rows with only zeros and no ones
print(len(df_pr[(df_pr == 0).all(axis=1)]))
# drop rows with only zeros
df_pr = df_pr[~(df_pr == 0).all(axis=1)]
df_pr.info()

2
<class 'pandas.core.frame.DataFrame'>
Index: 82 entries, bed dimensions to harvested yield
Data columns (total 32 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   trial_001  82 non-null     float64
 1   trial_006  82 non-null     float64
 2   trial_008  82 non-null     float64
 3   trial_009  82 non-null     float64
 4   trial_011  82 non-null     float64
 5   trial_013  82 non-null     float64
 6   trial_015  82 non-null     float64
 7   trial_016  82 non-null     float64
 8   trial_018  82 non-null     float64
 9   trial_023  82 non-null     float64
 10  trial_026  82 non-null     float64
 11  trial_034  82 non-null     float64
 12  trial_036  82 non-null     float64
 13  trial_040  82 non-null     float64
 14  trial_044  82 non-null     float64
 15  trial_045  82 non-null     float64
 16  trial_048  82 non-null     float64
 17  trial_052  82 non-null     float64
 18  trial_055  82 non-null     float64
 19  trial_059  82 non-null     fl

In [56]:
# Read in the data dict csv file
df_dict = pd.read_csv('data_dict.csv')
print(df_dict.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   attribute  84 non-null     object
 1   entity     84 non-null     object
dtypes: object(2)
memory usage: 1.4+ KB
None


In [57]:
# merge the dataframes
df_pr['entity'] = df_pr.index.map(df_dict.set_index('attribute')['entity'])
df_pr.sort_values(by=['entity', 'code'], inplace=True)
print(df_pr.info())

<class 'pandas.core.frame.DataFrame'>
Index: 82 entries, bed dimensions to harvested yield
Data columns (total 33 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   trial_001  82 non-null     float64
 1   trial_006  82 non-null     float64
 2   trial_008  82 non-null     float64
 3   trial_009  82 non-null     float64
 4   trial_011  82 non-null     float64
 5   trial_013  82 non-null     float64
 6   trial_015  82 non-null     float64
 7   trial_016  82 non-null     float64
 8   trial_018  82 non-null     float64
 9   trial_023  82 non-null     float64
 10  trial_026  82 non-null     float64
 11  trial_034  82 non-null     float64
 12  trial_036  82 non-null     float64
 13  trial_040  82 non-null     float64
 14  trial_044  82 non-null     float64
 15  trial_045  82 non-null     float64
 16  trial_048  82 non-null     float64
 17  trial_052  82 non-null     float64
 18  trial_055  82 non-null     float64
 19  trial_059  82 non-null     floa

In [58]:
# send to csv file
df_pr.to_csv('codes_all_processed.csv')

In [59]:
# Load trial directory
tdf = pd.read_csv('trials.csv', usecols=['Trial ID', 'Sample', 'Species', 'Trial Type'], dtype=str)
# Drop rows with missing values for the Sample column (unsampled trials)
tdf.dropna(subset=['Sample'], inplace=True)
tdf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32 entries, 0 to 93
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Trial ID    32 non-null     object
 1   Sample      32 non-null     object
 2   Species     32 non-null     object
 3   Trial Type  32 non-null     object
dtypes: object(4)
memory usage: 1.2+ KB


In [60]:
# Create a dataframe with the codes and the fraction of trials in which they appear
df_pr.drop(columns=['entity'], inplace=True)
sum_df = df_pr.sum(axis=1)
sum_df = sum_df/tdf.shape[0]
sum_df = sum_df.to_frame()
sum_df.columns = ['All_Trials']
# Set decimal precision to 2
sum_df = sum_df.round(2)

In [61]:
# For the subset of trials with each species, add a column that indicates the fraction of trials in which each code appears
species = ['Pepper', 'Tomato', 'Watermelon']
for s in species:
    # Get the subset of trials that include the species substring in the Species column
    subset = tdf.loc[tdf['Species'].str.contains(s)]['Trial ID'].to_list()
    # Subset cdf to include columns from the subset of trials
    df_subset = df_pr[[col for col in df_pr.columns if col.split('_')[1] in subset]]
    # Calculate the fraction of trials in which each code appears
    sum_df[s] = df_subset.sum(axis=1)/len(subset)
    # Set decimal precision to 2
    sum_df[s] = sum_df[s].round(2)

# For the subset of trials with each trial type, add a column that indicates the fraction of trials in which each code appears
trial_types = ['Management', 'Variety']
for t in trial_types:
    # Get the subset of trials that include the trial type substring in the Trial Type column
    subset = tdf.loc[tdf['Trial Type'].str.contains(t)]['Trial ID'].to_list()
    # Subset cdf to include columns from the subset of trials
    df_subset = df_pr[[col for col in df_pr.columns if col.split('_')[1] in subset]]
    # Calculate the fraction of trials in which each code appears
    sum_df[t] = df_subset.sum(axis=1)/len(subset)
    # Set decimal precision to 2
    sum_df[t] = sum_df[t].round(2)
print(sum_df)

                                   All_Trials  Pepper  Tomato  Watermelon  \
code                                                                        
bed dimensions                           0.84    0.92    0.75        0.86   
rows per bed                             0.38    0.69    0.25        0.00   
cultivar growth pattern                  0.16    0.00    0.42        0.00   
cultivar name                            1.00    1.00    1.00        1.00   
fertilizer application date              0.75    0.77    0.75        0.71   
...                                       ...     ...     ...         ...   
weed control method                      0.62    0.62    0.67        0.57   
cumulative harvested fruit number        0.53    0.38    0.58        0.71   
cumulative harvested yield               0.81    0.92    0.75        0.71   
harvested fruit number                   0.09    0.00    0.08        0.29   
harvested yield                          0.09    0.00    0.08        0.29   

In [62]:
# add entity to sum_df using code and attribute from sum_df and ddf
sum_df['entity'] = sum_df.index.map(df_dict.set_index('attribute')['entity'])

# save the dataframe to a csv file
sum_df.to_csv('codes_summary_processed.csv', index=True)

## Scoring Parameter Set Derivation
Now that the parameters have been processed, it is necessary to extract the subset of parameters that will be used for dataset/model scoring. All parameters which are not used in any model will be removed, as will parameters which are synonymous in practical terms. For the purpose of defining a parameter set for models, it can be assumed that no variety trial data contains soil horizonal or profile data beyond what can be analyzed in the root zone, via a soil test. Furthermore, weather parameters will not be considered unless explicitly measured. The selection will occur manually.